In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings

warnings.filterwarnings('ignore')

In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'size_width_cookware'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Plates & Platters"""

# regex_pattern = r'(?i)(Measures\s?\d*.?\d*.?\")|(\s\d*.?.?Inch)|(\s\d*\s?\d*.?\d*.?"\sW)'

regex_pattern = r'(?i)(\s\d+.?.?Inch)|(\s\d+\s?\d*.?\d*.?"\sW)|(\s\d+\s?\d*.?\d*.?\s?diameter)|(Measures\s?\d+.?\d*.?"\s?diameter)|(Measures\s?\d+.?\d*.?")|(\d+.?\"\s?.?.?\s?diameter)'


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
metric = ' Inches'

range_string = """
 0 - 2.9
 12 - 14.9
 15 - 17.9
 18 - 20.9
 21 or Greater
 3 - 5.9
 6 - 8.9
 9 - 11.9
 """

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
df

product_id external_id  \
0     1213281732    69686315   
1      988484855    69597686   
2      817164556    43267903   
3      817153737    11194648   
4      817469184    47697355   
...          ...         ...   
6313  1012041299    11050476   
6314   817163424    42714804   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                               product_name  \
0                                        NFL Philadelphia Eagles Aluminum Patterned Coaster   
1     Zwilling® J.A. Henckels Clad CFX Ceramic Nonstick Stainless Steel 2-Piece Fry Pan Set   
2                                         Zwilling J.A. Henckels Aurora 8-Inch Open Fry Pan   
3                                         Farberware® Classic Nonstick 12-Inch Deep Skillet   
4             Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
...                                                                                     ...   
6313                                               Luminarc 11.25-Inch Stackable Glass Bowl   
6314                                                 Fiesta® 4-Piece Place Setting in Poppy   
6315                                                    Fiesta® Luncheon Plate in Turquoise   
6316                                                          Fiesta® Dinner Plate in Lapis   
6317                                                        Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
1                                                       

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0" - 2.9"',
 '12" - 14.9"',
 '15" - 17.9"',
 '18" - 20.9"',
 '21" or Greater',
 '3" - 5.9"',
 '6" - 8.9"',
 '9" - 11.9"',
 'n/a']

In [7]:
kwp=KeywordProcessor()
kwp.add_keywords_from_list(list_values)
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

0       []
1       []
2       []
3       []
4       []
        ..
6313    []
6314    []
6315    []
6316    []
6317    []
Name: long_desc, Length: 6318, dtype: object

In [8]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [9]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=df[df['matches'].astype(str)!='[]']

In [10]:
x[0:100]

product_id external_id  \
0    1213281732    69686315   
4     817469184    47697355   
5     817476141    62441551   
6     817476158    62441964   
7     817498227    41016350   
8     817555221    64713052   
9     817631351    69487325   
10    817636220    69517014   
11    988487679    69624176   
13    817471970    61005426   
14   1007044173    69634980   
15   1070130951    69634988   
16   1156991209    17140353   
17   1143303850    43965625   
18   1162593918    69691619   
19   1110713531    69514005   
20   1159730107    69691612   
21   1087371535    69648086   
22   1099397540    62441841   
23   1065850077    69648119   
24   1018170434    69635293   
25   1010396275    69620854   
26   1005128244    69625779   
27    997140804    67730995   
28   1005130703    69634973   
29    997149448    67898190   
30    997140805    67731008   
31    988772391    16938203   
32    988513028    46318169   
33    988667126    77477977   
34    988485219    44040031   
35    817479110    62976442   
36    817479104    62976282   
37    817193111    76497963   
38   1219063828    63034868   
39   1219063828    63034868   
42    817190531    62869126   
43    817187307    62358927   
45    817186030    62055840   
46    817185980    62018258   
47    817185587    61870994   
48    817183857    61460836   
49    817183849    61460621   
51    817471286    60782281   
52    817180643    60782342   
53    817179496    60432186   
56    817175369    47218086   
57    817204945    47170971   
58   1162894683    47147461   
67    817201368    45583957   
70    817462984    45373251   
71    817463166    45478840   
73    817195077    44959944   
74    817462028    44511739   
75    817167435    44511562   
77   1203399509    44321147   
78    817460897    43921430   
79    817200089    43626199   
80    817200064    43626083   
81    817460359    43597789   
82    817200058    43551552   
83    817164206    43106264   
84    817459684    43082575   
85   1038308675    43082124   
86    817165450    43781690   
88    817458839    42427056   
89    817162954    42411482   
91    817161944    41750254   
93    817458718    42276203   
96    817162224    41937716   
97      3745823    42075790   
98    817458290    41876091   
99      3745798    42075547   
102   817160123    40522609   
103   817160104    40518220   
104   817159633    40194158   
105   817456548    40190969   
106   817158173    18903520   
107   817158149    18903458   
108   817455575    18903385   
109   817158164    18901803   
110   817455563    18901706   
111   817158158    18901633   
112   817158154    18901595   
113   817455557    18901579   
114   817482616    18882442   
115   817450903    18881535   
116   817482594    18900467   
118   817455242    18544385   
120   817157699    18540827   
122   817454679    17551060   
123   817157479    18411164   
125   817661067    17140027   
126   817454126    16443352   
127   817480290    15161710   
129   817454279    16750700   
130   817155150    14554041   
132   817452812    12822030   
133   817154350    13011931   
134   817162249    12999496   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
8                                    13-Inch Scalloped Charger Plates in Silver (Set of 6)   
9                   kate spade new york Vintage Cherry Dot Tidbit Plates in Red (Set of 4)   
10                           Certified International Sweet Bunny Dessert Plates (Set of 4)   
11                  Certifi

In [11]:
x

product_id external_id  \
0     1213281732    69686315   
4      817469184    47697355   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
...          ...         ...   
6312   817554472    76982646   
6313  1012041299    11050476   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
...                                                                                    ...   
6312  Q Squared Hampton Toile 5.5-Inch Round Bread & Butter Plate (Set of 4) in Blue/White   
6313                                              Luminarc 11.25-Inch Stackable Glass Bowl   
6315                                                   Fiesta® Luncheon Plate in Turquoise   
6316                                                         Fiesta® Dinner Plate in Lapis   
6317                                                       Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                    Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Inject modern flair into your tablescape with

In [12]:
# x['matches'].explode().value_counts()[0:100]

In [13]:
# df[df['matches'].astype(str)=='[]']

In [14]:
# x[x['value'].astype(str)!='Greater than 30 Inches'][100:150]

In [15]:
# df[df['external_id'].astype(str)=='00739441']

In [16]:
# x['matches'].explode().value_counts()

In [17]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [18]:
split_col(x)

In [19]:
# x['matches'].explode().value_counts()

In [20]:
error

NameError: name 'error' is not defined

In [21]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\"\-','',str(measure)))
x

product_id external_id  \
0     1213281732    69686315   
4      817469184    47697355   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
...          ...         ...   
6312   817554472    76982646   
6313  1012041299    11050476   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
...                                                                                    ...   
6312  Q Squared Hampton Toile 5.5-Inch Round Bread & Butter Plate (Set of 4) in Blue/White   
6313                                              Luminarc 11.25-Inch Stackable Glass Bowl   
6315                                                   Fiesta® Luncheon Plate in Turquoise   
6316                                                         Fiesta® Dinner Plate in Lapis   
6317                                                       Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                    Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Inject modern flair into your tablescape with

In [22]:
def remove_fractions(numbers):
    if numbers.find('/') > -1:
#         numbers = literal_eval(numbers)
#         inte, fract = numbers.split('-')
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
#         num = int(inte) + tmp
        return str(tmp)
    else:
        return numbers

In [23]:
# x['matches'].explode().value_counts()

In [27]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

product_id external_id  \
0     1213281732    69686315   
4      817469184    47697355   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
...          ...         ...   
6312   817554472    76982646   
6313  1012041299    11050476   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
...                                                                                    ...   
6312  Q Squared Hampton Toile 5.5-Inch Round Bread & Butter Plate (Set of 4) in Blue/White   
6313                                              Luminarc 11.25-Inch Stackable Glass Bowl   
6315                                                   Fiesta® Luncheon Plate in Turquoise   
6316                                                         Fiesta® Dinner Plate in Lapis   
6317                                                       Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                    Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Inject modern flair into your tablescape with

In [26]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'\"','',str(measure)))
x

product_id external_id  \
0     1213281732    69686315   
4      817469184    47697355   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
...          ...         ...   
6312   817554472    76982646   
6313  1012041299    11050476   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
...                                                                                    ...   
6312  Q Squared Hampton Toile 5.5-Inch Round Bread & Butter Plate (Set of 4) in Blue/White   
6313                                              Luminarc 11.25-Inch Stackable Glass Bowl   
6315                                                   Fiesta® Luncheon Plate in Turquoise   
6316                                                         Fiesta® Dinner Plate in Lapis   
6317                                                       Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                    Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Inject modern flair into your tablescape with

In [35]:
x=x[x['matches'].astype(str)!="10'"]
x=x[x['matches'].astype(str)!="12''"] 
x=x[x['matches'].astype(str)!="612”"]   
x=x[x['matches'].astype(str)!="6½"]



In [33]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r'\-','',str(measure)))

In [36]:
x['matches'].explode().value_counts()

9         352
11        241
6         238
8.25      215
13        190
10.5      181
8         177
8.5       168
12        145
6.5       127
10.75     119
14        104
7          99
7.5        85
16         83
10         82
6.25       71
6.75       68
4          67
5          66
9.5        62
12.5       55
9.75       54
7.25       45
8.75       43
9.25       38
11.75      38
10.25      36
4.5        36
23.13      35
11.25      31
5.75       30
12.25      30
11.5       30
2          27
4.25       25
5.5        24
15         24
3          23
13.5       22
25         20
20.25      18
3.75       18
1.5        16
1          16
15.5       15
12.75      14
13.6       14
14.5       14
102        13
4.75       13
13.75      13
3.5        12
3.3        12
54         12
13.25      12
7.75       11
15.75      11
75         11
19         11
6.3        11
5.25       10
1.25        9
10.2        9
20          9
18          9
9.125       8
8.3         8
3.76        7
6.63        7
17          7
6.69  

In [ ]:
# x[x['matches'].astype(str)=='47218086'] #47218086

In [50]:
x=x[(x['matches'].astype(float)>1)&(x['matches'].astype(float)<100)]

In [51]:
x['matches']=x['matches'].astype(float)

In [52]:
x

product_id external_id  \
0     1213281732    69686315   
4      817469184    47697355   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
...          ...         ...   
6312   817554472    76982646   
6313  1012041299    11050476   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
...                                                                                    ...   
6312  Q Squared Hampton Toile 5.5-Inch Round Bread & Butter Plate (Set of 4) in Blue/White   
6313                                              Luminarc 11.25-Inch Stackable Glass Bowl   
6315                                                   Fiesta® Luncheon Plate in Turquoise   
6316                                                         Fiesta® Dinner Plate in Lapis   
6317                                                       Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                    Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Inject modern flair into your tablescape with

# matches

In [53]:
matches=x
matches

product_id external_id  \
0     1213281732    69686315   
4      817469184    47697355   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
...          ...         ...   
6312   817554472    76982646   
6313  1012041299    11050476   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
...                                                                                    ...   
6312  Q Squared Hampton Toile 5.5-Inch Round Bread & Butter Plate (Set of 4) in Blue/White   
6313                                              Luminarc 11.25-Inch Stackable Glass Bowl   
6315                                                   Fiesta® Luncheon Plate in Turquoise   
6316                                                         Fiesta® Dinner Plate in Lapis   
6317                                                       Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                    Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Inject modern flair into your tablescape with

In [54]:
matches['matches']=matches['matches'].astype(float)

In [55]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  0 and num <= 2.9:
        return " 0 - 2.9 Inches"
        
    if num >=  12 and num <= 14.9:
        return " 12 - 14.9 Inches"
        
    if num >=  15 and num <= 17.9:
        return " 15 - 17.9 Inches"
        
    if num >=  18 and num <= 20.9:
        return " 18 - 20.9 Inches"
        
    if num >=  3 and num <= 5.9:
        return " 3 - 5.9 Inches"
        
    if num >=  6 and num <= 8.9:
        return " 6 - 8.9 Inches"
        
    if num >=  9 and num <= 11.9:
        return " 9 - 11.9 Inches"
        
    else:
        return r"n/a"



In [56]:
# matches[matches['value'].astype(str)=='Greater than 50 Inches']

In [57]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

product_id external_id  \
0     1213281732    69686315   
4      817469184    47697355   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
...          ...         ...   
6312   817554472    76982646   
6313  1012041299    11050476   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
...                                                                                    ...   
6312  Q Squared Hampton Toile 5.5-Inch Round Bread & Butter Plate (Set of 4) in Blue/White   
6313                                              Luminarc 11.25-Inch Stackable Glass Bowl   
6315                                                   Fiesta® Luncheon Plate in Turquoise   
6316                                                         Fiesta® Dinner Plate in Lapis   
6317                                                       Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                    Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Inject modern flair into your tablescape with

In [58]:
matches['Q:size_width_cookware'].explode().value_counts()

 6 - 8.9 Inches      1487
 9 - 11.9 Inches     1368
 12 - 14.9 Inches     638
 3 - 5.9 Inches       401
 15 - 17.9 Inches     160
n/a                    96
 0 - 2.9 Inches        68
 18 - 20.9 Inches      59
Name: Q:size_width_cookware, dtype: int64

In [59]:
matches=matches[matches['Q:size_width_cookware'].astype(str)!='n/a']

In [60]:
matches

product_id external_id  \
0     1213281732    69686315   
4      817469184    47697355   
5      817476141    62441551   
6      817476158    62441964   
7      817498227    41016350   
...          ...         ...   
6312   817554472    76982646   
6313  1012041299    11050476   
6315   817156716    17337190   
6316   817457830    41502136   
6317   817454382    16946389   

                                                                              product_name  \
0                                       NFL Philadelphia Eagles Aluminum Patterned Coaster   
4            Fortessa Fortaluxe Super White OJO 12.25-Inch Rectangular Platters (Set of 4)   
5                                              Vista Alegre Carré Dinner Plates (Set of 4)   
6                             Vista Alegre Mar 7.75-Inch Bread and Butter Plate (Set of 4)   
7                                Noritake® Colorwave Mini Plates in Green Apple (Set of 4)   
...                                                                                    ...   
6312  Q Squared Hampton Toile 5.5-Inch Round Bread & Butter Plate (Set of 4) in Blue/White   
6313                                              Luminarc 11.25-Inch Stackable Glass Bowl   
6315                                                   Fiesta® Luncheon Plate in Turquoise   
6316                                                         Fiesta® Dinner Plate in Lapis   
6317                                                       Fiesta® Dinner Plate in Scarlet   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                    Crafted by the artisans of Wendell August Forge, the NFL Aluminum Patterned Coaster features your team's name and helmet expertly engraved in solid aluminum. This year-round coaster is perfect for gameday fun as well as everyday home decor. Show your team pride with this unique symbol of two great American traditions, professional football and hospitality Pattern depicts your favorite team's name and helmet Forged by the artisans of Wendell August Forge Makes a great gift for any fan 100% aluminum Wipe clean Measures 4.5\" diameter Weighs 0.55 lb. Officially licensed by the NFL Made in USA   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    Inject modern flair into your tablescape with

In [61]:
matches['Q:size_width_cookware'].explode().value_counts()

 6 - 8.9 Inches      1487
 9 - 11.9 Inches     1368
 12 - 14.9 Inches     638
 3 - 5.9 Inches       401
 15 - 17.9 Inches     160
 0 - 2.9 Inches        68
 18 - 20.9 Inches      59
Name: Q:size_width_cookware, dtype: int64

# fixed_na

In [ ]:
fixed_na=matches[matches['Q:outside_diameter'].astype(str)=='n/a']

In [ ]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [ ]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

In [ ]:
fixed_na['Q:outside_diameter'].explode().value_counts()

# Wipe

In [ ]:
wipe=df[['external_id']].astype(str)
wipe['Q:outside_diameter']=''
wipe

# Null Values

In [ ]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [ ]:
na_external_id

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [62]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/BedBathandBeyond - {attribute} - match upload_Shawn.csv',index=False)

In [63]:
looks_good('Bed Bath & Beyond', attribute, df, matches)